In [ ]:
!./ngrok authtoken <Your Token>


In [1]:
!pip install flask-ngrok
!pip install pydub
!pip install gTTS
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 34.7 MB/s eta 0:00:00


In [ ]:

pip install Flask torch librosa transformers gtts moviepy pyngrok
import os
from flask import Flask, request, render_template_string, send_from_directory
from moviepy.editor import VideoFileClip
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from gtts import gTTS
from pyngrok import ngrok  # REPLACEMENT for flask_ngrok

# Set folders
UPLOAD_FOLDER = 'uploads'
OUTPUT_FOLDER = 'outputs'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Initialize Flask app
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Load Whisper model
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.eval()

# Transcribe function
def transcribe_audio(audio_file_path):
    audio_input, _ = librosa.load(audio_file_path, sr=16000, mono=True)
    inputs = processor(audio_input, return_tensors="pt", sampling_rate=16000)
    with torch.no_grad():
        predicted_ids = model.generate(inputs["input_features"], max_length=1024)
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

# Dummy emotion detection
def detect_emotion(transcribed_text):
    emotions = ["neutral", "happy", "sad", "angry"]
    return "neutral", 0.9  # Placeholder

# Dummy stammer severity
def detect_stammer_severity(audio_file_path):
    return "mild"  # Placeholder

# Convert to fluent audio
def convert_to_fluent_audio(text, wav_file_path):
    tts = gTTS(text=text, lang='en')
    mp3_file_path = os.path.join(OUTPUT_FOLDER, os.path.basename(wav_file_path).replace(".wav", "_fluent.mp3"))
    tts.save(mp3_file_path)
    return mp3_file_path

# Home route
@app.route('/')
def index():
    return render_template_string('''
        <!doctype html>
        <html>
        <head>
            <title>Speech Analyzer</title>
            <style>
                body { font-family: Arial; padding: 20px; }
                .container { max-width: 600px; margin: auto; }
                input[type="file"] { margin: 10px 0; }
                .btn { padding: 10px 15px; background-color: #4CAF50; color: white; border: none; cursor: pointer; }
            </style>
        </head>
        <body>
            <div class="container">
                <h2>Upload Audio/Video for Analysis</h2>
                <form action="/upload" method="post" enctype="multipart/form-data">
                    <input type="file" name="file" required>
                    <button type="submit" class="btn">Upload</button>
                </form>
            </div>
        </body>
        </html>
    ''')

# Serve output files
@app.route('/outputs/<filename>')
def download_file(filename):
    return send_from_directory(OUTPUT_FOLDER, filename)

# Upload route
@app.route('/upload', methods=['POST'])
def upload_audio():
    if 'file' not in request.files:
        return "No file uploaded"

    file = request.files['file']
    filename = file.filename
    file_path = os.path.join(UPLOAD_FOLDER, filename)
    file.save(file_path)

    # Convert video to audio if needed
    if filename.lower().endswith(('.mp4', '.mkv', '.avi', '.mov')):
        audio_path = os.path.join(UPLOAD_FOLDER, filename.rsplit('.', 1)[0] + ".wav")
        clip = VideoFileClip(file_path)
        clip.audio.write_audiofile(audio_path)
        file_path = audio_path

    # Ensure file exists
    if not os.path.exists(file_path):
        return "File conversion failed."

    # Transcribe and analyze
    audio_text = transcribe_audio(file_path)
    emotion, score = detect_emotion(audio_text)
    severity = detect_stammer_severity(file_path)
    fluent_audio = convert_to_fluent_audio(audio_text, file_path)
    fluent_audio_filename = os.path.basename(fluent_audio)

    return render_template_string('''
        <!doctype html>
        <html>
        <head>
            <title>Results</title>
            <style>
                body { font-family: Arial; padding: 20px; }
                .container { max-width: 700px; margin: auto; }
                .btn { padding: 10px 15px; background-color: #008CBA; color: white; border: none; text-decoration: none; }
            </style>
        </head>
        <body>
            <div class="container">
                <h2>Transcription Result</h2>
                <p><strong>Text:</strong> {{ audio_text }}</p>
                <p><strong>Detected Emotion:</strong> {{ emotion }} (Confidence: {{ confidence }})</p>
                <p><strong>Stammer Severity:</strong> {{ severity }}</p>
                <p><strong>Fluent Audio:</strong></p>
                <audio controls>
                    <source src="/outputs/{{ fluent_audio_filename }}" type="audio/mpeg">
                    Your browser does not support the audio element.
                </audio><br><br>
                <a href="/outputs/{{ fluent_audio_filename }}" class="btn" download>Download Fluent Audio</a>
            </div>
        </body>
        </html>
    ''', audio_text=audio_text, emotion=emotion, confidence=round(score, 2),
         severity=severity, fluent_audio_filename=fluent_audio_filename)

# Start the app and ngrok tunnel
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"\n * ngrok tunnel available at: {public_url.public_url}\n")
    app.run()
